## Deploy SDXL as Online Endpoint on Azure Machine Learning
This notebook was developed and tested on an Azure ML Compute Instance using the `Python 3.10 - SDK V2` Jupyter kernel. 

In [1]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, Model, Environment, CodeConfiguration, OnlineRequestSettings 
from azure.identity import DefaultAzureCredential

# enter details of your Azure Machine Learning workspace
subscription_id = "<Your Azure subscription ID>"
resource_group = "<Name of the Resource Group that contains your Azure ML resource>"
workspace = "<Name of your Azure ML Workspace>"

In [2]:
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

computes = ml_client.compute.list()

In [3]:
# Define an endpoint name
endpoint_name = "image-gen"

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name, 
    description="Stable Diffusion Models",
    auth_mode="key"
)

ml_client.online_endpoints.begin_create_or_update(endpoint)

In [6]:
endpoint.scoring_uri

In [7]:
from azure.ai.ml.constants import AssetTypes
model_name = "sd-dummy" # dummy model for deployment. The actual SDXL model will automatically be downloaded by the scoring script

model = Model(
    path="assets/dummy-model.txt",
    type=AssetTypes.CUSTOM_MODEL,
    name=model_name,
    description="A dummy model for SD deployment.",
)
ml_client.models.create_or_update(model)

Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'sd-dummy', 'description': 'A dummy model for SD deployment.', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/92645b1b-8a8a-4693-b8e2-c214a523fe40/resourceGroups/energy/providers/Microsoft.MachineLearningServices/workspaces/energy/models/sd-dummy/versions/13', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/akopp1/code/Users/akopp', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f50a93d9f60>, 'serialize': <msrest.serialization.Serializer object at 0x7f50a93d95d0>, 'version': '13', 'latest_version': None, 'path': 'azureml://subscriptions/92645b1b-8a8a-4693-b8e2-c214a523fe40/resourceGroups/energy/workspaces/energy/datastores/workspaceblobstore/paths/LocalUpload/5f9c2a69a8e7b55a6cac71baa26db1b6/dummy-model.txt', 'datastore': None, 'utc_tim

In [8]:
env = Environment(
    conda_file="assets/conda.yaml",
    image="mcr.microsoft.com/azureml/curated/minimal-ubuntu20.04-py38-cuda11.6.2-gpu-inference:24",
)

In [ ]:
req_timeout = OnlineRequestSettings(request_timeout_ms=90000, max_queue_wait_ms=900000)

deployment = ManagedOnlineDeployment(
    name='sdxl',
    endpoint_name=endpoint_name,
    model=model,
    environment= env,
    code_configuration = CodeConfiguration(code='assets', scoring_script='score.py'),
    instance_type = 'Standard_NC24ads_A100_v4',
    instance_count=1,
    request_settings=req_timeout
)

ml_client.online_deployments.begin_create_or_update(deployment)

In [20]:
deployment

ManagedOnlineDeployment({'private_network_connection': None, 'provisioning_state': None, 'endpoint_name': 'energy-sd', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/akopp-gpu/code/Users/akopp', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7fef04176e60>, 'model': '/subscriptions/92645b1b-8a8a-4693-b8e2-c214a523fe40/resourceGroups/energy/providers/Microsoft.MachineLearningServices/workspaces/energy/models/sd-dummy/versions/1', 'code_configuration': {'code': '/subscriptions/92645b1b-8a8a-4693-b8e2-c214a523fe40/resourceGroups/energy/providers/Microsoft.MachineLearningServices/workspaces/energy/codes/14f82306-2ec7-4346-87d5-29515858b5aa/versions/1'}, 'environment': '/subscriptions/92645b1b-8a8a-4693-b8e2-c214a523fe40/resourceGroups/energy/providers/Microsoft.MachineLearningServices/works

....................................................................................................................................................................................................................................................................................................................